 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [ ]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
from enum import Enum
from tabulate import tabulate
from math import nan, isnan

##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number

pd.options.display.float_format = '{:.3f}'.format



In [ ]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [ ]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [ ]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False #--> GUI with active modes and phases, start condition [active modes list ] as bool (true false). Generarte excel which generete flow chart.


In [ ]:
if import_mode: 
    ##Change working directory to temp/epic
    #os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


In [ ]:
#print(os.listdir(cwd)) #List directory

#Random_mode:
Generate aleatories PID numbers and symbols.



In [ ]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [ ]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

PID.xlsx

In [ ]:
#Load PID data
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')


In [ ]:
#Create a list from df column

#Add to other dataframe
#Using series
#pid_series = pd.Series(df_data_excel['Prop_PID_Number'])
#df_data_excel = df_data_excel.assign(sPID = pid_series)

#From a list
#1st list to a dataframe, then join
#df_pid_list = pd.DataFrame(pid_list)
#df_element_list = df_data_excel.join(df_pid_list)
#or Append the list directly to the DataFrame:
#df_data_excel['sPID'] = pid_list

In [ ]:
df_data_excel = df_data_excel.sort_values('Prop_PID_Element') #Sorting by Element we get elements ordered and ready to compare
if verbose_mode: print(df_data_excel)

In [ ]:
df_data_excel

In [ ]:
#Remove ShapeKey from the dataframe
del df_data_excel['ShapeKey']

In [ ]:
# dropping duplicate values
df_data_excel.drop_duplicates(inplace=True)

Drop elements like filters, catalizators, etc

In [ ]:
banned_elements = ['B','F', 'C', 'D', 'S', 'W', 'Z']
df_data_excel = df_data_excel[~df_data_excel['Prop_PID_Element'].isin(banned_elements)].reindex()

AutomationCode.xlsx

In [ ]:
#Load Automation Code excel file
df_automation_code = pd.read_excel('AutomationCode.xlsx')
if debug_mode: df_automation_code.head()

TND.xlsx

In [ ]:
#Load Tag Name Definition
tnd_pid = pd.read_excel('TND.xlsx')
#tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
if debug_mode: tnd_pid

Functions definitions

In [ ]:
# Def of Lookup as in Microsoft Excel.
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [ ]:
# Lookup for PID number: 1.840 or 2.840 return the same definition: Diff. pressure sensor
def xlookupPID(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    lookup_value = str("%.3f" % lookup_value)
    lookup_value = 'Y' + lookup_value[1:]
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [ ]:
#Group by PID number
element_list = df_data_excel.groupby(['Prop_PID_Number']).agg(list).copy()



Group Filter
#Drop compound elements which are not part of the software list
#Drop elements like filters, catalizators, etc

In [ ]:
#Group filter: 
banned_elements = [['PI', 'V'],['ANOTHER', 'COMBO']]
element_list = element_list[~element_list['Prop_PID_Element'].isin(banned_elements)].reindex()

Reset the dataframe index. Objects can be easily accessed

In [ ]:
##Add tag number definition - reset index first, then use xlookup function
element_list = element_list.reset_index()

In [ ]:
#list of float to string conversion
#spid_list = [str("%.3f" % pid) for pid in pid_list]
pid_list = list((element_list['Prop_PID_Number']))
spid_list = ['{:.3f}'.format(pid) for pid in pid_list]
element_list['sPID'] = spid_list

In [ ]:
element_list

Deletion of elements that are non electronics, like manual valves.

In [ ]:
#Deletion of manual valves, as we have no other information in the PID
#TODO: Use Y.xxx pattern
#TODO: import from a file banned element
#TODO: save deleted elements in a file

banned_elements = [1.654,1.656,1.850,1.863]
element_list = element_list[~element_list['Prop_PID_Number'].isin(banned_elements)].reindex()

In [ ]:
#Add Tag Number definition
element_list['Tag_Number_Definition'] = element_list['Prop_PID_Number'].apply(xlookupPID, args = (tnd_pid['PID'], tnd_pid['Description']));

In [ ]:
if debug_mode: 
    print(tabulate(element_list, headers = 'keys', tablefmt = 'psql'))

df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)
          temp_c  temp_f
Portland    17.0    62.6
Berkeley    25.0    77.0
You can create multiple columns within the same assign where one of the columns depends on another one defined within the same assign:

>>> df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
...           temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)
          temp_c  temp_f  temp_k
Portland    17.0    62.6  290.15
Berkeley    25.0    77.0  298.15

In [ ]:
#Add columns
#element_list_SWMs = ['SWM','SWMNr','Cx1','Cx1Nr','Cx1','Cx1Nr', 'Cx1','Cx1Nr','Cx1','Cx1Nr','Element_Description']
element_list_SWMs = ['SWM','SWM_Nr','Cx1','Cx1_Nr','Cx2','Cx2_Nr', 'Cx3','Cx3_Nr','Cx4','Cx4_Nr','Element_Description']
element_list = element_list.assign(**dict.fromkeys(element_list_SWMs))

Add elements that has two software main modules, as 1.825 Leak valve (motor,reg.limit) and 1.825 Leak (leak, limit)
SIS module and gasket are now possible to configure

index+1     1.825   [Leak] [Limit_hi_lo] 

In [ ]:
#TODO: use patterns (Y.XXX) instead
if (1.655 and 1.680) in element_list.Prop_PID_Number.values: 
    dicc_AHU01 = {'Prop_PID_Number': 'AHU01', 'Prop_PID_Element': 'AHU01' , 'Tag_Number_Definition': 'Leader/Follower AHU 01 1.800 - (1.655-1.680'}
    element_list = pd.concat([element_list, pd.DataFrame([dicc_AHU01])])

if (1.800) in element_list.Prop_PID_Number.values:
    dicc_1800 = {'Prop_PID_Number': '1.800_precon', 'Prop_PID_Element': '[702] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_deco', 'Prop_PID_Element': '[118] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_pro', 'Prop_PID_Element': '[114] [15°C < 21°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_aer1', 'Prop_PID_Element': '[708] [30°C < 40°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

if (1.810) in element_list.Prop_PID_Number.values:
    dicc_1810 = {'Prop_PID_Number': '1.810_pro', 'Prop_PID_Element': '[114] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])

    dicc_1810 = {'Prop_PID_Number': '1.810_precon', 'Prop_PID_Element': '[702] [15%RH < 20%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])

    dicc_1810 = {'Prop_PID_Number': '1.810_deco', 'Prop_PID_Element': '[118] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])


if (1.820) in element_list.Prop_PID_Number.values:
    dicc_1820 = {'Prop_PID_Number': '1.820_pro', 'Prop_PID_Element': '[114] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

    dicc_1820 = {'Prop_PID_Number': '1.820_deco', 'Prop_PID_Element': '[118] [> 0.20m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

    dicc_1820 = {'Prop_PID_Number': '1.820_aer2', 'Prop_PID_Element': '[709] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

#LEAK: S_LEAK with Limit hi lo, which limit hi lo reads how much flow is used and generates alarm Leak too high.
if (1.825) in element_list.Prop_PID_Number.values:
    dicc_1825 = {'Prop_PID_Number': '1.825_leak', 'Prop_PID_Element': 'LEAK' , 'Tag_Number_Definition': 'Flow rate sensor'}
    #element_list = element_list.append(dicc_1825, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1825])])

if (1.840) in element_list.Prop_PID_Number.values:
    dicc_1840 = {'Prop_PID_Number': '1.840_prod', 'Prop_PID_Element': '[114] [>30Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_deco', 'Prop_PID_Element': '[118] [>15Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_leak', 'Prop_PID_Element': '[108] [60Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_emy', 'Prop_PID_Element': '[105] [<-200Pa >200Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])


##Analog error goes also here
#Like 1.550 analog fault, 1.555



In [ ]:
#CREATE DOSING UNITS
#if (1.815) in element_list.Prop_PID_Number.values:
#    dicc_sis_1 = {'Prop_PID_Number': 'DU01', 'Prop_PID_Element': 'DOSING_UNIT' , 'Tag_Number_Definition': 'Dosing unit 01'}
#    #element_list = element_list.append(dicc_1840, ignore_index=True)
#    element_list = pd.concat([element_list, pd.DataFrame([dicc_sis_1])])

#if (1.816) in element_list.Prop_PID_Number.values:
#    dicc_sis_2 = {'Prop_PID_Number': 'DU02', 'Prop_PID_Element': 'DOSING_UNIT' , 'Tag_Number_Definition': 'Dosing unit 02'}
#    #element_list = element_list.append(dicc_1840, ignore_index=True)
#    element_list = pd.concat([element_list, pd.DataFrame([dicc_sis_2])])

In [ ]:
element_list.reset_index(drop=True, inplace=True)

In [ ]:
#Software Module Definition
SOFTWARE_MODE_DEFINITION = {
    "S_AAS_MAS_TCP" : "S_AAS_MAS_TCP",
    "S_AAS_MODBUS_TCP" : "S_AAS_MODBUS_TCP",
    "S_AAS_SERIAL" : "S_AAS_SERIAL",
    "S_ALARM" : "S_ALARM",
    "S_DRIVE" : "S_DRIVE",
    "S_ELMT_ON_OFF" : "S_ELMT_ON_OFF",
    "S_EMITTER_CMT" : "S_EMITTER_CMT",
    "S_FIPA" : "S_FIPA",
    "S_FLOW_METER" : "S_FLOW_METER",
    "S_GASKET" : "S_GASKET",
    "S_LEAK" : "S_LEAK",
    "S_LIMIT_HI_LO" : "S_LIMIT_HI_LO",
    "S_LIMIT_ON_OFF" : "S_LIMIT_ON_OFF",
    "S_LIMIT_SWITCH" : "S_LIMIT_SWITCH",
    "S_LINE" : "S_LINE",
    "S_MOTOR1" : "S_MOTOR1",
    "S_PLANT" : "S_PLANT",
    "S_PMS_MODBUS_ASCII" : "S_PMS_MODBUS_ASCII",
    "S_PMS_MODBUS_TCP" : "S_PMS_MODBUS_TCP",
    "S_PUMP_TEST" : "S_PUMP_TEST",
    "S_REG_CONT" : "S_REG_CONT",
    #S_REG = "S_REG"
    "S_SIS" : "S_SIS",
    "S_SKANFOG" : "SKANFOG",
    #S_TIMESYNCH = "S_TIMESYNCH"
    "S_VALVE_FLAP" : "S_VALVE_FLAP",
    #S_WEIGHT_SCALE_NG = "S_WEIGHT_SCALE_NG"
    #S_WEIGHT_SCALE = "S_WEIGHT_SCALE"
    "S_WEIGHT_SCALE2_NG" : "S_WEIGHT_SCALE2_NG"
    #"S_LEADER_FOLLOWER" : "S_LEADER_FOLLOWER"
    }

In [ ]:
#Application Module definition
APPLICATION_MODULE_DEFINITION = {
    "A_SERVO_DRIVE" : "A_SERVO_DRIVE"
}

In [ ]:
#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
    #response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SWM_Nr']
    #print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
    #index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
    #print(index_of[0]) #[0] first element, [-1] last element

    #element_list.SWM_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
    #element_list.Prop_PID_Number[indice] = 'Change pid number'
    #element_list.Cx3_Nr[indice] = 11
#using index
    #element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

#SOFT SORTING. Classifies objects just for the combination of PID elements

In [ ]:
#SOFT SORTING. Classifies objects just for the combination of PID elements

#get value in one line of code:
#element_list.at[items, 'Cx3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SWM_Nr']
#Other way:
#element_list.Cx3_Nr[items] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SWM_Nr']

#Counters:
idx_drive = 0
idx_motor = 0
idx_reg = 0
idx_limit_hi_lo = 0
idx_limit_on_off = 0
idx_elmt_on_off = 0
idx_valve_flap = 0
idx_limit_switch = 0
sis = 0
sis_pump = 0
weight_scale2_ng = 0
leak = 0
particle_counter = 0
s_aas_mas_tcp = 0
s_fipa = 0
#leader_follower = 0
air_dryer = 0
idx_a_servo_drive = 0
idx_pump_nr = 0



for items in element_list.index:

    #yzzz = str("%.3f" % element_list.sPID[items])
    #yzzz = element_list.sPID[items]
    #Xzzz = yzzz[0]
    #coma = yzzz[1]
    #yXzz = yzzz[2]
    #yXXz = yzzz[2:4]
    #yXXX = yzzz[2:5]

    #Xzzz = int(float(yzzz[0]))
    #coma = yzzz[1]
    #yXzz = int(float(yzzz[2]))
    #yXXz = int(float(yzzz[2:4]))
    #yXXX = int(float(yzzz[2:5]))

    yzzz = element_list.sPID[items]
    #Check if SPID is an string element
    if isinstance(yzzz, str):
        Xzzz = yzzz[0]
        #coma = yzzz[1]
        yXzz = yzzz[2]
        yXXz = yzzz[2:4]
        yXXX = yzzz[2:5]
        #print(yzzz)




    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Drive (3 Cxs)
        case ['M', 'SC', 'TSA+', 'V']:
            #Drive
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_DRIVE']
            idx_drive = idx_drive + 1 
            element_list.at[items, 'SWM_Nr'] = idx_drive

            #Motor Conn 1
            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_motor = idx_motor + 1
            element_list.at[items, 'Cx1_Nr'] = idx_motor

            #Reg Conn 2
            element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            idx_reg = idx_reg + 1
            element_list.at[items, 'Cx2_Nr'] = idx_reg

            #Limit hi lo conn3
            element_list.at[items, 'Cx3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Cx3_Nr'] = '?'
  
        #Motor/reg/limit hi lo  (2 Cxs)
        case ['H', 'M', 'NC'] | ['EC', 'TSA+'] | ['FIA+', 'H', 'YC']:
            #['H', 'M', 'NC']: AHU Cooling
            #['EC', 'TSA+'] : AHU heating
            #['FIA+', 'H', 'YC']: Leak ventil control
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_motor = idx_motor + 1
            element_list.at[items, 'SWM_Nr'] = idx_motor

            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            idx_reg = idx_reg + 1
            element_list.at[items, 'Cx1_Nr'] = idx_reg

            element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Cx2_Nr'] = '?'

        #Motor Simple
        case ['M', 'SC', 'V', 'XA'] | ['V', 'YC']: 
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_motor = idx_motor + 1
            element_list.at[items, 'SWM_Nr'] = idx_motor

        #Motorized valve (uses limit hi lo for analog fault)
        case ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC']:
            #Original element get main function (in this case motor)
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_motor = idx_motor + 1
            element_list.at[items, 'SWM_Nr'] = idx_motor
            #A limit hi lo is first assigned for the analog fault
            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Cx1_Nr'] = idx_limit_hi_lo

            #Create the limit hi lo for the analog fault with same number assigned, not sure it can be created here....
                    
            dicc_limit_hi_lo = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'] , 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"], 
                'SWM_Nr'                : idx_limit_hi_lo,                          #Same given to Gasket.Cx1_Nr
                'Cx1'                   : SOFTWARE_MODE_DEFINITION['S_MOTOR1'],     #Same given to Gasket.SWM
                'Cx1_Nr'                : idx_motor,                               #Same given to Gasket.SWM_Nr
                }
            element_list = pd.concat([element_list, pd.DataFrame([dicc_limit_hi_lo])])

        

        #Limit hi lo
        case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TIA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo
            #Plus analog fault limit hi lo??

        #idx_elmt_on_off
        case ['M', 'P'] | ['L']:  
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_ELMT_ON_OFF']
            idx_elmt_on_off = idx_elmt_on_off + 1
            element_list.at[items, 'SWM_Nr'] = idx_elmt_on_off

        #Some valves are more complicated and uses LImit on off
        case ['V']:
            if int(yXXX) == 860:
                element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_ON_OFF']
                idx_limit_on_off = idx_limit_on_off + 1
                element_list.at[items, 'SWM_Nr'] = idx_limit_on_off

                #Limit hi lo conn1
                element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
                element_list.at[items, 'Cx1_Nr'] = '?'
                
            else:
                element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_VALVE_FLAP']
                idx_valve_flap = idx_valve_flap + 1
                element_list.at[items, 'SWM_Nr'] = idx_valve_flap
        
        #Airdryer
        case ['T', 'XA']:
            #element_list.at[items, 'SWM'] = 'Air dryer Munter or similar'
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_ON_OFF']
            idx_limit_on_off = idx_limit_on_off + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_on_off

            #Limit hi lo conn1
            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Cx1_Nr'] = '?'


        case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_VALVE_FLAP']
            idx_valve_flap = idx_valve_flap + 1
            element_list.at[items, 'SWM_Nr'] = idx_valve_flap

        #idx_limit_switch
        case ['FSA-'] | ['GZA-'] | ['GSA+'] | ['GSA-'] | ['LSA-'] : 
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_SWITCH']
            idx_limit_switch = idx_limit_switch + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_switch

        #Gasket
        case ['PSA-', 'V']:
            if int(yXXX) >= 865:
                print('Gasket are created according to how many inputs are available') 
            else:
                print('To avoid same elements that are not connected to gaskets:', element_list.at[items, 'sPID'])
                element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_SWITCH']
                idx_limit_switch = idx_limit_switch + 1
                element_list.at[items, 'SWM_Nr'] = idx_limit_switch

        #Servo mousehole
        case ['GIA', 'M', 'SC', 'XA']:
            #Limit hi lo conn1
            element_list.at[items, 'SWM'] = APPLICATION_MODULE_DEFINITION["A_SERVO_DRIVE"]
            idx_a_servo_drive = idx_a_servo_drive + 1
            element_list.at[items, 'SWM_Nr'] = idx_a_servo_drive
        
        #Particle counter
        case ['FA-', 'QIA+', 'XA']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_PMS_MODBUS_TCP']
            particle_counter = particle_counter + 1
            element_list.at[items, 'SWM_Nr'] = particle_counter


        #Air Sampler MAS TCP
        case ['P', 'XA']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_AAS_MAS_TCP']
            s_aas_mas_tcp = s_aas_mas_tcp + 1
            element_list.at[items, 'SWM_Nr'] = s_aas_mas_tcp


        #FiPa
        case ['M', 'XA']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_FIPA']
            s_fipa = s_fipa + 1
            element_list.at[items, 'SWM_Nr'] = s_fipa


        ## Artifcial sensors
        case 'LEAK': #Uses Y.825
            #Limit hi lo conn1
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo #Festo 

            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION['S_LEAK']
            leak = leak + 1
            element_list.at[items, 'Cx1_Nr'] = leak

        case '[702] [17°C < 28°C]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo
        
        case '[118] [17°C < 28°C]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[114] [15°C < 21°C]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[708] [30°C < 40°C]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[114] [< 100%RH]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[702] [15%RH < 20%RH]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[118] [< 100%RH]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[114] [> 0.45m/s]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[118] [> 0.20m/s]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[709] [> 0.45m/s]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[114] [>30Pa]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[118] [>15Pa]': 
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo
        
        case '[108] [60Pa]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case '[105] [<-200Pa >200Pa]':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SWM_Nr'] = idx_limit_hi_lo

        case 'AHU01':
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            idx_reg = idx_reg + 1
            element_list.at[items, 'SWM_Nr'] = idx_reg
            
            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            #idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Cx1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SWM_Nr']

            element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #idx_reg = idx_reg + 1
            element_list.at[items, 'Cx2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.655].iloc[0]['Cx1_Nr']

            element_list.at[items, 'Cx3'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #idx_reg = idx_reg + 1
            element_list.at[items, 'Cx3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.680].iloc[0]['Cx1_Nr']


            #print("Yikers. I don't know that one.")

############################################################################################################################


        #SIS pumps: Controlled though SIS module
        #['M', 'P', 'SC']    Y.7x0       S_SIS               TBD *1
        case ['M', 'P', 'SC']:

            #New assignemt of pumps ti S_PUMP_TEST
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION["S_PUMP_TEST"] #APLICATION MODULE PUMP (CONTOL, DIRECTION, SPEED) #SOFTWARE_MODE_DEFINITION["S_PUMP_TEST"]
            #Asigned by repartition later                        
            element_list.at[items, 'SWM_Nr'] = '?' #sis_pump_airflow #sis_pump_700 for heating plates, sis_pump_skanfog with H2O2 flow sensor (without weight scale)

            element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_SIS"]
            element_list.at[items, 'Cx1_Nr'] = '?'

            element_list.at[items, 'Cx2'] = 'pump_nr'
            idx_pump_nr = idx_pump_nr + 1
            element_list.at[items, 'Cx2_Nr'] = idx_pump_nr

            element_list.at[items, 'Cx3'] = 'Pump[x]'
            element_list.at[items, 'Cx3_Nr'] = '?'  

        #WICA
        #['WICA±']           Y.81x WIKA  S_WEIGHT_SCALE2_NG  +1           A_DOSING_UNIT      SWM_Nr=1    S_LIMIT_HI_LO   +1 
        case ['WICA±']:
            element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_WEIGHT_SCALE2_NG']
            weight_scale2_ng = weight_scale2_ng + 1
            element_list.at[items, 'SWM_Nr'] = weight_scale2_ng

            #Cx1: DOSING_UNIT
            element_list.at[items, 'Cx1'] = 'A_DOSING_UNIT'
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Cx1_Nr'] = weight_scale2_ng

            #S_LIMIT_HI_LO conn1
            element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Cx2_Nr'] = idx_limit_hi_lo




            
            #List of connected pumps
            #element_list.at[items, 'Cx5'] = SOFTWARE_MODE_DEFINITION["A_SIS_PUMP"]
            #idx_limit_hi_lo = idx_limit_hi_lo + 1
            #element_list.at[items, 'Cx5_Nr'] = '?'      

############################################################################################################################


In [ ]:
print(tabulate(element_list, headers = 'keys', tablefmt = 'psql'))

Creation of Dosing Unit, SIS, SIS_Mode (700 or A), SKANFOG, Pump Test, Pump list, nozzle list, heating plate list and Gasket sensor list

1. Detection by Prop_PID_Element
                        PID         ['SWM',             'SWM_Nr', 'Cx1',             'Cx1_Nr',  'Cx2',            'Cx2_Nr',     'Cx3',             'Cx3_Nr',  'Cx4',           'Cx4_Nr',   'Element_Description']
1.1 ['WICA±']           Y.81x WIKA  S_WEIGHT_SCALE2_NG  +1         A_DOSING_UNIT      WSWM_Nr   S_LIMIT_HI_LO     +1 

1.1 APPEND and UPDATE VALUES OF ?:
                    Y.81x       S_SIS               WSWM_Nr   A_DOSING_UNIT      WSWM_Nr   S_LIMIT_HI_LO(W)  WIKA:Cx2_Nr   S_LIMIT_HI_LO(T)   ?         S_LIMIT_HI_LO(H)  ?
                    
                    Y.81x       S_PUMP_TEST         WSWM_Nr   A_DOSING_UNIT      WSWM_Nr    

                    Y.81x       A_DOSING_UNIT       WSWM_Nr   S_WEIGHT_SCALE2_NG WSWM_Nr   S_SIS             WSWM_Nr      S_PUMP_TEST        WSWM_Nr 

*WSWM_Nr = WIKA SOFTWARE MODULE Nr = every ['WICA±'] increments his number, and every sis, pump_test or dosing units get this number.


2. Detection by Prop_PID_Element
    2.1 ['M', 'P', 'SC']    Y.7x0       S_SIS            TBD *1    Cx1 = A_PUMPS      Cx1_Nr (given)
    2.2 TBD *1: (SUGGESTION: (Y.7x0 is SIS Y) AND PUMP_LIST UNITS / SIS UNITS 6/2 = 3 each SIS)

3. CHECK IF FILE A_PUMP_LIST.xlsx exists, if not, create. Modify, run again and config from file is loaded.

OPTIONAL:
                    #Y.7x0       A_PUMP              1(inc)
                    #PUMPL       A_PUMP_LIST         SWM_Nr=1       S_SIS           TBD *1      S_PUMP_TEST    TBD *1                                             
                    #NZZL        A_NOZZLE_LIST
                    #HPLT        A_HEATING_PLATE_LIST





Y.86x       S_GASKET            1(inc)       S_LIMIT_SWITCH ?           A_GASKET_LIST   S_GASKET=1  ['PSA-', 'V']
Y.87X       S_GASKET            1(inc)       S_LIMIT_SWITCH ?           A_GASKET_LIST   S_GASKET=1

2. APPEND:
                    Y.81x       S_SIS               SWM_Nr=1     A_DOSING_UNIT      SWM_Nr=1    S_LIMIT_HI_LO(T)    ?        S_LIMIT_HI_LO(H)     ?          S_LIMIT_HI_LO(W)  ?
                    
                    Y.81x       S_PUMP_TEST         SWM_Nr=1     A_DOSING_UNIT      SWM_Nr=1    

                    Y.81x       A_DOSING_UNIT       SWM_Nr=1     S_WEIGHT_SCALE2_NG SWM_Nr=1    S_SIS            SWM_Nr=1    S_PUMP_TEST SWM_Nr=1 

In [ ]:
#APPEND: WICA±

for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
            case ['WICA±']:
                #Y.81x       S_SIS               SWM_Nr=1     A_DOSING_UNIT      SWM_Nr=1    S_LIMIT_HI_LO(W)    WIKA:Cx2_Nr      S_LIMIT_HI_LO(T)     ?           S_LIMIT_HI_LO(H)  ?
                #print("I have found a WICA element in index: ",items)
                #print(element_list.at[items, 'Prop_PID_Number'])
                #print(element_list.at[items, 'Prop_PID_Element'])
                #print(element_list.at[items, 'SWM'])
                #print(element_list.at[items, 'SWM_Nr'])

                #Temperature isolator
                #If element is in Y.81x, get info from Y.800 -> 2.816 -> 2.800
                y800 = str("%.3f" % element_list.Prop_PID_Number[items])
                y800 = float(y800[0] + '.800')
                #print(element_list[element_list['Prop_PID_Number'] == y800].iloc[0]['SWM_Nr'])

                #Humidity isolator
                #If element is in Y.81x, get info from Y.810 -> 2.816 -> 2.810
                y810 = str("%.3f" % element_list.Prop_PID_Number[items])
                y810 = float(y810[0] + '.810')
                #print(element_list[element_list['Prop_PID_Number'] == y810].iloc[0]['SWM_Nr'])


                dicc_sis = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], #Same as wika
                'Prop_PID_Element'      : '[SIS]' , 
                'Tag_Number_Definition' : 'SIS MODULE', 
                'SWM'                   : 'S_SIS', 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],     #WIKA:SWM_Nr
                'Cx1'                   : element_list.at[items, 'Cx1'],        #dosing unit
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],     #dosing unit number
                'Cx2'                   : element_list.at[items, 'Cx2'],        #limit hi lo of wika
                'Cx2_Nr'                : element_list.at[items, 'Cx2_Nr'],     #limit hi lo of wika number
                'Cx3'                   : 'S_LIMIT_HI_LO',                      #limit hi lo of Isolator Temperature 
                'Cx3_Nr'                : element_list[element_list['Prop_PID_Number'] == y800].iloc[0]['SWM_Nr'],  #TODO: Y.81x --> >.800 
                'Cx4'                   : 'S_LIMIT_HI_LO',                      #limit hi lo of Isolator Humidity    
                'Cx4_Nr'                : element_list[element_list['Prop_PID_Number'] == y810].iloc[0]['SWM_Nr'] #TODO: Y.81x --> >.810 
                }
                #element_list = pd.concat([element_list, pd.DataFrame([dicc_sis])])

                #Y.81x       S_PUMP_TEST         SWM_Nr=1     A_DOSING_UNIT      SWM_Nr=1
                dicc_pump_test = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], #Same as wika
                'Prop_PID_Element'      : '[PTEST]' , 
                'Tag_Number_Definition' : 'PUMP TEST MODULE', 
                'SWM'                   : 'S_PUMP_TEST', 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],     #WIKA:SWM_Nr
                'Cx1'                   : element_list.at[items, 'Cx1'],        #dosing unit
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],     #dosing unit number
                }
                #element_list = pd.concat([element_list, pd.DataFrame([dicc_pump_test])])

                #Y.81x       A_DOSING_UNIT       SWM_Nr=1   S_WEIGHT_SCALE2_NG SWM_Nr=1   S_SIS             SWM_Nr=1      S_PUMP_TEST        SWM_Nr=1 
                
                dicc_dosing_unit = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], #Same as wika
                'Prop_PID_Element'      : '[DU]' , 
                'Tag_Number_Definition' : 'DOSING UNIT MODULE', 
                'SWM'                   : element_list.at[items, 'Cx1'],        #WIKA:Cx1 --> A_DOSING_UNIT
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     #WIKA:Cx1_Nr
                'Cx1'                   : 'S_WEIGHT_SCALE2_NG',                 #S_WEIGHT_SCALE2_NG
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],     #WIKA:SWM_Nr
                'Cx2'                   : 'S_SIS',                              #S_SIS
                'Cx2_Nr'                : element_list.at[items, 'SWM_Nr'],     #WIKA:SWM_Nr
                'Cx3'                   : 'S_PUMP_TEST',                        #S_PUMP_TEST 
                'Cx3_Nr'                : element_list.at[items, 'SWM_Nr']      #WIKA:SWM_Nr
                }
                #element_list = pd.concat([element_list, pd.DataFrame([dicc_dosing_unit])])

Assing limit switch to gasket (Door, Service cover, mousehole, etc)

In [ ]:
MAX_SAFETY_DOORS_PER_GASKET = 8
MAX_SERVICE_COVERS_PER_GASKET = 8
MAX_MOUSEHOLES_DOORS_PER_GASKET = 1.5

idx_gasket = 0
idx_gasket_safety_doors = 0
idx_gasket_service_covers = 0
idx_gasket_mousehole_doors = 0

#Safety doors
list_safety_doors_with_nan= element_list['SWM_Nr'].where(element_list['Tag_Number_Definition'] == 'Safety door').to_list()
#list_safety_doors_with_nan
list_safety_doors_without_nan = [x for x in list_safety_doors_with_nan if isnan(x) == False]
amount_safety_doors = len(list_safety_doors_without_nan)

#Safety Service cover
list_service_cover_with_nan= element_list['SWM_Nr'].where(element_list['Tag_Number_Definition'] == 'Service cover').to_list()
#list_safety_doors_with_nan
list_service_cover_without_nan = [x for x in list_service_cover_with_nan if isnan(x) == False]
amount_service_covers = len(list_service_cover_without_nan)

#Mousehole door
list_mousehole_door_with_nan= element_list['SWM_Nr'].where(element_list['Tag_Number_Definition'] == 'Mousehole door').to_list()
#list_safety_doors_with_nan
list_mousehole_door_without_nan = [x for x in list_mousehole_door_with_nan if isnan(x) == False]
amount_mousehole_doors = len(list_mousehole_door_without_nan)

#######################################################
print("Amount of safety doors: ",amount_safety_doors)
print("Amount of service cover: ",amount_service_covers)
print("Amount of mousehole doors: ",amount_mousehole_doors)

needed_gasket_safety_doors = int((amount_safety_doors/MAX_SAFETY_DOORS_PER_GASKET) + 0.9)
needed_gasket_service_cover = int((amount_service_covers/MAX_SERVICE_COVERS_PER_GASKET) + 0.9)
needed_gasket_mousehole_doors = int((amount_mousehole_doors/MAX_MOUSEHOLES_DOORS_PER_GASKET) + 0.9)

print('Gaskets for safety doors needed: ',needed_gasket_safety_doors)
print('Gaskets for service cover needed: ',needed_gasket_service_cover)
print('Gaskets for mousehole doors needed: ',needed_gasket_mousehole_doors)

#for pumps in list_pump_without_nan:
if amount_safety_doors > MAX_SAFETY_DOORS_PER_GASKET:
    print('Error: has exceeded the maximum number of safety door per gasket') 
if amount_service_covers > MAX_SERVICE_COVERS_PER_GASKET:
    print('Error: has exceeded the maximum number of service covers per gasket')
if amount_mousehole_doors > MAX_MOUSEHOLES_DOORS_PER_GASKET:
    print('Error: has exceeded the maximum number of mousehole doors per gasket')

#Assign inputs to Gaskets

#Safety doors gaskets
ratio_safety_doors_gasket = (amount_safety_doors / needed_gasket_safety_doors)
list_safety_doors_limit_switch_index = element_list.index.where(element_list['Tag_Number_Definition'] == 'Safety door').to_list()
list_safety_doors_limit_switch_index = [x for x in list_safety_doors_limit_switch_index if isnan(x) == False]

for items in list_safety_doors_limit_switch_index:
    #First numeration in Cx2
    element_list.at[items, 'Cx2'] = "A_GASKET_SAFETY_DOOR"
    idx_gasket_safety_doors = idx_gasket_safety_doors + 1
    element_list.at[items, 'Cx2_Nr'] = idx_gasket_safety_doors
    #Second assign to gasket number
    element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_GASKET"]
    element_list.at[items, 'Cx1_Nr'] = int((idx_gasket_safety_doors/ratio_safety_doors_gasket) +0.9)
    last_safety_door_gasket_used = int((idx_gasket_safety_doors/ratio_safety_doors_gasket) +0.9)

gasket_used = last_safety_door_gasket_used

#Service cover gaskets
#idx_gasket_service_covers = needed_gasket_safety_doors #Initilize with the last gasket number in use
ratio_service_cover_gasket = (amount_service_covers / needed_gasket_service_cover)
list_service_cover_limit_switch_index = element_list.index.where(element_list['Tag_Number_Definition'] == 'Service cover').to_list()
list_service_cover_limit_switch_index = [x for x in list_service_cover_limit_switch_index if isnan(x) == False]

print('last safety door gasket used: ', gasket_used)

for items in list_service_cover_limit_switch_index:
    #First numeration in Cx2
    element_list.at[items, 'Cx2'] = "A_GASKET_SERVICE_COVER"
    idx_gasket_service_covers = idx_gasket_service_covers + 1
    element_list.at[items, 'Cx2_Nr'] = idx_gasket_service_covers
    #Second assign to gasket number in Cx1
    element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_GASKET"]
    element_list.at[items, 'Cx1_Nr'] = int((idx_gasket_service_covers/ratio_service_cover_gasket) +0.9) + gasket_used
    last_service_cover_gasket_used = int((idx_gasket_service_covers/ratio_service_cover_gasket) +0.9)
    

gasket_used = last_service_cover_gasket_used + last_safety_door_gasket_used

#Mousehole 
#idx_gasket_mousehole_doors = needed_gasket_service_cover #Initilize with the last gasket number in use
ratio_mousehole_doors_gasket = (amount_mousehole_doors / needed_gasket_mousehole_doors)
list_mousehole_doors_limit_switch_index = element_list.index.where(element_list['Tag_Number_Definition'] == 'Mousehole door').to_list()
list_mousehole_doors_limit_switch_index = [x for x in list_mousehole_doors_limit_switch_index if isnan(x) == False]

print('last gasket used: ',gasket_used)

for items in list_mousehole_doors_limit_switch_index:
    #First numeration in Cx2
    element_list.at[items, 'Cx2'] = "A_GASKET_MOUSEHOLE_DOOR"
    idx_gasket_mousehole_doors = idx_gasket_mousehole_doors + 1
    element_list.at[items, 'Cx2_Nr'] = idx_gasket_mousehole_doors
    #Second assign to gasket number in Cx1
    element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_GASKET"]
    element_list.at[items, 'Cx1_Nr'] = int((idx_gasket_mousehole_doors/ratio_mousehole_doors_gasket) +0.9) + gasket_used
    last_mousehole_doors_gasket_used = int((idx_gasket_mousehole_doors/ratio_mousehole_doors_gasket) +0.9)


gasket_used = last_service_cover_gasket_used + last_safety_door_gasket_used + last_mousehole_doors_gasket_used
print('last gasket used: ',gasket_used)

##Update Gasket anc create s_limit_hi_lo
for items in element_list.index:
    yzzz = element_list.sPID[items]
    if isinstance(yzzz, str):
        Xzzz = yzzz[0]
        #coma = yzzz[1]
        yXzz = yzzz[2]
        yXXz = yzzz[2:4]
        yXXX = yzzz[2:5]
        #print(yzzz)


    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
        #Gasket
        case ['PSA-', 'V']:
            if int(yXXX) >= 865:
                element_list.at[items, 'SWM'] = SOFTWARE_MODE_DEFINITION['S_GASKET']
                idx_gasket = idx_gasket + 1
                element_list.at[items, 'SWM_Nr'] = idx_gasket
                #Third define a new limit hi lo and add to the element_list
                element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
                idx_limit_hi_lo = idx_limit_hi_lo + 1
                element_list.at[items, 'Cx1_Nr'] = idx_limit_hi_lo

                ##Add limit_hi_lo (will be an 1.865 SW S_GASKET and other as 1.865 S_LIMIT_HI_LO), to element_list
                dicc_limit_hi_lo = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'] , 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"], 
                'SWM_Nr'                : idx_limit_hi_lo,                          #Same given to Gasket.Cx1_Nr
                'Cx1'                   : SOFTWARE_MODE_DEFINITION['S_GASKET'],     #Same given to Gasket.SWM
                'Cx1_Nr'                : idx_gasket,                               #Same given to Gasket.SWM_Nr
                }
                element_list = pd.concat([element_list, pd.DataFrame([dicc_limit_hi_lo])])
            else:
                print('Not modified:', element_list.at[items, 'sPID'])


In [ ]:
element_list_a_gasket = element_list[element_list['SWM'] == 'S_GASKET']
ga_gasket = element_list_a_gasket.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
ga_gasket.first()

Assing pumps to SIS equitative

In [ ]:
#APPEND: Pumps belongs to SIS = ?
#Assing pumps to SIS equitative
MAX_PUMPS_IN_SIS = 6
list_sis_with_nan= element_list['SWM_Nr'].where(element_list['SWM'] == 'S_WEIGHT_SCALE2_NG').to_list()
list_sis_without_nan = [x for x in list_sis_with_nan if isnan(x) == False]
#print(list_sis_with_nan)
print(list_sis_without_nan)
nsis = len(list_sis_without_nan)

#New code
list_pump_with_nan= element_list['Cx2_Nr'].where(element_list['SWM'] == 'S_PUMP_TEST').to_list()
#list_pump_with_nan= element_list['Cx1_Nr'].where(element_list['Cx1'] == 'A_PUMPS').to_list()
list_pump_without_nan = [x for x in list_pump_with_nan if isnan(x) == False]
#print(list_pump_with_nan)
print(list_pump_without_nan)
npumps = len(list_pump_without_nan)

print(nsis)
print(npumps)
ratio = (npumps/nsis)
#print(ratio)

#for pumps in list_pump_without_nan:
if npumps/nsis > MAX_PUMPS_IN_SIS:
    print('Error: max number of pump per sis (6) has reached limit') 

for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
            case ['M', 'P', 'SC']:
                belongs_to_sis = int(((element_list.at[items, 'Cx2_Nr']) / ratio) + 0.9)
                element_list.at[items, 'SWM_Nr'] = belongs_to_sis
                element_list.at[items, 'Cx1_Nr'] = belongs_to_sis
                print(belongs_to_sis)

In [ ]:
#Reindex after add elements
#element_list = element_list.reset_index() Keeps the old index...
element_list.reset_index(drop=True, inplace=True)

In [ ]:
print(tabulate(element_list, headers = 'keys', tablefmt = 'psql'))

In [ ]:
#Analog fault - Limit hi lo adding
for items in element_list.index:
    
    yzzz = element_list.sPID[items]
    #Check if SPID is an string element
    if isinstance(yzzz, str):
        Xzzz = yzzz[0]
        #coma = yzzz[1]
        yXzz = yzzz[2]
        yXXz = yzzz[2:4]
        yXXX = yzzz[2:5]
        #print(yzzz)

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Motorized valve (uses limit hi lo for analog fault)
        case ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC']:
            #Create the limit hi lo for the analog fault 
            dicc_limit_hi_lo = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'] , 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],        #Limit hi lo already assigned in the original element (uses main function motor)
                'Cx1'                   : SOFTWARE_MODE_DEFINITION['S_MOTOR1'],    #Motor of motorized valve 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],         
                }
            element_list = pd.concat([element_list, pd.DataFrame([dicc_limit_hi_lo])])

In [ ]:
#Works if PID is the index
#value =(element_list.at[1.840, 'SWM_Nr'])
#print(value)

In [ ]:
#UPDATE with the S_limit hi lo value
for items in element_list.index:
    
    yzzz = element_list.sPID[items]
    #Check if SPID is an string element
    if isinstance(yzzz, str):
        Xzzz = yzzz[0]
        #coma = yzzz[1]
        yXzz = yzzz[2]
        yXXz = yzzz[2:4]
        yXXX = yzzz[2:5]
        #print(yzzz)

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Drive
        case ['M', 'SC', 'TSA+', 'V']:
            #Limit hi lo conn3
            element_list.at[items, 'Cx3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]

            #If element is 1.150, get info from 1.840, 2.150 get info from 2.840
            y840 = str("%.3f" % element_list.Prop_PID_Number[items])
            y840 = float(y840[0] + '.840')
            element_list.Cx3_Nr[items] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SWM_Nr']
            #or
            #element_list.at[items, 'Cx3_Nr'] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SWM_Nr']

            #Y.840 #only when PID is index
            #element_list.at[items, 'Cx3_Nr'] = (element_list.at[y840, 'SWM_Nr']) 
            

        #1.655 [H, M, NC] Control valve (Cool water control)
        case ['H', 'M', 'NC']: 
            #element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Cx2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SWM_Nr']

        #Some valves are more complicated and uses LImit on off
        case ['V']:
            if int(yXXX) == 860:
                #Limit hi lo conn1
                #element_list.at[items, 'Cx1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
                y840 = str("%.3f" % element_list.Prop_PID_Number[items])
                y840 = float(y840[0] + '.840')
                element_list.at[items, 'Cx1_Nr'] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SWM_Nr']
        #Air dryer
        case ['T', 'XA']:
            y810 = str("%.3f" % element_list.Prop_PID_Number[items])
            y810 = float(y810[0] + '.810')
            element_list.Cx1_Nr[items] = element_list[element_list['Prop_PID_Number'] == y810].iloc[0]['SWM_Nr'] #Humidity sensor Y.810

        #1.680 [D, EC, TSA+]  Electric heating
        case ['EC', 'TSA+']:
            #element_list.at[items, 'Cx2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Cx2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SWM_Nr']

        #Leak control
        case ['FIA+', 'H', 'YC']:
            element_list.at[items, 'Cx2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SWM_Nr']

        case 'AHU01':
            element_list.at[items, 'Cx1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SWM_Nr']
            element_list.at[items, 'Cx2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.655].iloc[0]['Cx1_Nr']
            element_list.at[items, 'Cx3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.680].iloc[0]['Cx1_Nr']

        





In [ ]:
if debug_mode: 
    print(tabulate(element_list, headers = 'keys', tablefmt = 'psql'))

In [ ]:
#Export to excel

# create the writer object
writer = pd.ExcelWriter('Element_list_export.xlsx')
# write dataframe to excel
element_list.to_excel(writer)
# save the excel
writer.save()
# close excel
writer.close()


Init group element list

In [ ]:
#Create a new dataframe, group elements by binaries paar, later group by SWMs, SIS, Gasket, etc...
group_element_list = pd.DataFrame()
#group_element_list = pd.DataFrame(columns=['Prop_PID_Number','Prop_PID_Element','Tag_Number_Definition','SWM','SWM_Nr','Cx1','Cx1_Nr'])
group_element_list = pd.DataFrame(columns=['Prop_PID_Number','Prop_PID_Element','sPID','Tag_Number_Definition','SWM','SWM_Nr','Cx1','Cx1_Nr'])
if debug_mode: print(group_element_list)

#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
#response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SWM_Nr']
#print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
#print(index_of[0]) #[0] first element, [-1] last element

#element_list.SWM_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
#element_list.Prop_PID_Number[indice] = 'Change pid number'
#element_list.Cx3_Nr[indice] = 11
#using index
#element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

In [ ]:
#dicc_1825 = {'Prop_PID_Number': '1.825_leak', 'Prop_PID_Element': 'LEAK' , 'Tag_Number_Definition': 'Flow rate sensor'}
#element_list = element_list.append(dicc_1825, ignore_index=True)
#group_element_list = element_list.copy()    
#group_element_list.loc[:] = None
#group_element_list.iloc[0] = element_list.iloc[0]
#group_element_list.iloc[0] = element_list.iloc[1]
#group_element_list = pd.concat([group_element_list, element_list.loc[0]], axis="columns")
#print(element_list.iloc[1])

In [ ]:
group_element_list

In [ ]:
#LA MANDANGA
for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
        
        #Pids with NO Cx
        #Motor Simple
        #case ['M', 'SC', 'V', 'XA'] | ['V', 'YC'] | ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC'] | ['FIA+', 'H', 'YC']: 
            #group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #Limit hi lo
        #case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TIA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±'] | ['PSA-', 'V']:
            #group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #elmt_on_off
        #case ['M', 'P'] | ['L']:  
            #group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #valve_flap
        #case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y'] | ['V']:
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #limit_switch
        #case ['FSA-'] | ['GZA-'] | ['GSA+'] | ['GSA-'] | ['LSA-'] : 
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #Servo mousehole
        #case ['GIA+', 'GIA-', 'M', 'SC', 'XA']:
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1
        
        #Particle counter
        #case ['FA-', 'QIA+', 'XA']:
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #Air Sampler MAS TCP
        #case ['P', 'XA']:
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1

        #FiPa
        #case ['M', 'XA']:
        #    group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SWM : Cx1 

        case ['T', 'XA'] | ['M', 'P', 'SC'] | 'LEAK' | '[PTEST]':
        #case '[PTEST]':
            #['T', 'XA']: S_AIR_DRYER - (1)S_LIMIT_HI_LO
            dicc_01 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_02 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'SWM'],        
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],     
                }
                
            group_element_list = pd.concat([group_element_list, pd.DataFrame([dicc_01,dicc_02])])

        #Pids with 2 Cxs
        case ['H', 'M', 'NC'] | ['EC', 'TSA+'] | ['WICA±']:
            #['H', 'M', 'NC']:          S_REG -          (1) S_LIMIT_HI_LO - (2) S_REG_CONT - Cooling with Leader
            #['EC', 'TSA+']:            S_REG -          (1) S_LIMIT_HI_LO - (2) S_REG_CONT - heating with leader
            #['WICA±']:                 S_WEIGHT_SCALE - (1) S_SIS -         (2) S_PUMP_TEST 
            dicc_01 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_02 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            dicc_03 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'SWM'],        
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],     
                }
            dicc_04 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx2'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],     
                }
            dicc_05 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_06 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
                
            group_element_list = pd.concat([group_element_list, pd.DataFrame([dicc_01,dicc_02,dicc_03,dicc_04,dicc_05,dicc_06])])


        #Pids with 3 Cxs
        case ['M', 'SC', 'TSA+', 'V'] | 'AHU01':# | '[DU]' :
        #case ['M', 'SC', 'TSA+', 'V'] | 'AHU01' | '[DU]' :
            dicc_01 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_02 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            dicc_03 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx3'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],
                }
            
            dicc_04 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'SWM'],        
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],     
                }
            dicc_05 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx2'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],     
                }
            dicc_06 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx3'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],     
                }
            dicc_07 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_08 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_09 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx3'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],
                }

            dicc_10 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_11 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_12 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            group_element_list = pd.concat([group_element_list, pd.DataFrame([dicc_01,dicc_02,dicc_03,dicc_04,dicc_05,dicc_06,dicc_07,dicc_08,dicc_09,dicc_10,dicc_11,dicc_12])])


            
       #Pids with 4 Cxs
        #case 'DOSING_UNIT' | '[SIS]':
        case '[SIS]':
            #DOSING_UNIT: S_SIS - (1) s_LIMIT_HI_LO - (2)S_LIMIT_HI_LO - (3)S_WICA - (4)S_PUMP_TEST
            dicc_01 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_02 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            dicc_03 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx3'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],
                }
            dicc_04 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx4'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx4_Nr'],
                }
            dicc_05 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'SWM'],        
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],     
                }
            dicc_06 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx2'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],     
                }
            dicc_07 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx3'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],     
                }
            dicc_08 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'], 
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx1'], 
                'SWM_Nr'                : element_list.at[items, 'Cx1_Nr'],     
                'Cx1'                   : element_list.at[items, 'Cx4'],        
                'Cx1_Nr'                : element_list.at[items, 'Cx4_Nr'],     
                }
            dicc_09 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_10 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_11 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx3'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],
                }
            dicc_12 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx2'], 
                'SWM_Nr'                : element_list.at[items, 'Cx2_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx4'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx4_Nr'],
                }
            dicc_13 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_14 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_15 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            dicc_16 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx3'], 
                'SWM_Nr'                : element_list.at[items, 'Cx3_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx4'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx4_Nr'],
                }
            dicc_17 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx4'], 
                'SWM_Nr'                : element_list.at[items, 'Cx4_Nr'],
                'Cx1'                   : element_list.at[items, 'SWM'], 
                'Cx1_Nr'                : element_list.at[items, 'SWM_Nr'],
                }
            dicc_18 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx4'], 
                'SWM_Nr'                : element_list.at[items, 'Cx4_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            dicc_19 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx4'], 
                'SWM_Nr'                : element_list.at[items, 'Cx4_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx2'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx2_Nr'],
                }
            dicc_20 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'Cx4'], 
                'SWM_Nr'                : element_list.at[items, 'Cx4_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx3'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx3_Nr'],
                }
            
            #group_element_list = pd.concat([group_element_list, pd.DataFrame([dicc_01,dicc_02,dicc_03,dicc_04,dicc_05,dicc_06,
            #dicc_07,dicc_08,dicc_09,dicc_10,dicc_11,dicc_12,dicc_13,dicc_14,dicc_15,dicc_16,dicc_17,dicc_18,dicc_19,dicc_20])])
        

        case _:
            dicc_01 = {
                'Prop_PID_Number'       : element_list.at[items, 'Prop_PID_Number'],
                'Prop_PID_Element'      : element_list.at[items, 'Prop_PID_Element'], 
                'sPID'                  : element_list.at[items, 'sPID'], 
                'Tag_Number_Definition' : element_list.at[items, 'Tag_Number_Definition'], 
                'SWM'                   : element_list.at[items, 'SWM'], 
                'SWM_Nr'                : element_list.at[items, 'SWM_Nr'],
                'Cx1'                   : element_list.at[items, 'Cx1'], 
                'Cx1_Nr'                : element_list.at[items, 'Cx1_Nr'],
                }
            group_element_list = pd.concat([group_element_list, pd.DataFrame([dicc_01])])

    

        
 
      

           


In [ ]:
if debug_mode: 
    print(tabulate(element_list, headers = 'keys', tablefmt = 'psql'))

In [ ]:
#Reindex after add elements
#element_list = element_list.reset_index() Keeps the old index...
group_element_list.reset_index(drop=True, inplace=True)

In [ ]:
if debug_mode: 
    print(tabulate(group_element_list, headers = 'keys', tablefmt = 'psql'))

group_element_list

In [ ]:
#list of index positions where an element is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 1.150].to_list()
index_of

In [ ]:
#lenght of a list
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.150].to_list()

#print(len(index_of))
#print(index_of)

#element_list.loc[index_of].count(axis=1).max()

#element_list.notnull()


How to Get Cell Value from Pandas DataFrame?

dataframe[‘column_name’].loc[dataframe.index[row_number]]

In [ ]:
#Bring description of elements connected
#group_element_list['Element_Description'] = element_list['Prop_PID_Number'].apply(xlookup, args = (tnd_pid['PID'], tnd_pid['Description']));
#group_element_list['Element_Description'] = group_element_list['Tag_Number_Definition']

In [ ]:
#Delete unneccessary columns
#group_element_list = group_element_list.drop(['Prop_PID_Element','Cx2','Cx2_Nr','Cx3','Cx3_Nr','Cx4','Cx4_Nr'], axis=1)
#group_element_list = group_element_list.drop(['Prop_PID_Element','Cx2','Cx2_Nr','Cx3','Cx3_Nr','Cx4','Cx4_Nr,Element_description'], axis=1)
#group_element_list = group_element_list.drop(['Prop_PID_Element'], axis=1)

In [ ]:
if debug_mode: print(tabulate(group_element_list, headers = 'keys', tablefmt = 'psql'))

In [ ]:
#gdrive = group_element_list.groupby(['SWM','SWM_Nr','Prop_PID_Number','Tag_Number_Definition','Cx1','Cx1_Nr'])#.agg('Element_Description')
#,'Prop_PID_Number','Tag_Number_Definition','
gdrive = group_element_list.groupby(['SWM','SWM_Nr','sPID','Tag_Number_Definition','Cx1','Cx1_Nr'])
gdrive.first()

In [ ]:
gdrive.first().to_excel('GroupElementList.xlsx')

End _Group element list

In [ ]:
element_list.count()

In [ ]:
#Drives
element_list_drives = group_element_list[group_element_list['SWM'] == 'S_DRIVE']
gdrive = element_list_drives.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
gdrive.first()

In [ ]:
element_list_motors = group_element_list[group_element_list['SWM'] == 'S_MOTOR1']
gmotor = element_list_motors.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
gmotor.first()

In [ ]:
#AHU01
element_list_ahu01 = group_element_list[group_element_list['SWM'] == 'AHU01']
gahu01 = element_list_ahu01.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
gahu01.first()
    

In [ ]:
#Valve
element_list_valve_flap = group_element_list[group_element_list['SWM'] == 'S_VALVE_FLAP']
gvalve = element_list_valve_flap.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
gvalve.first()

In [ ]:
#Element on off
element_list_elmt_on_off = group_element_list[group_element_list['SWM'] == 'S_ELMT_ON_OFF']
gelmt_on_off = element_list_elmt_on_off.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
gelmt_on_off.first()

In [ ]:
element_list_limit_switch = group_element_list[group_element_list['SWM'] == 'S_LIMIT_SWITCH']
glimit_switch = element_list_limit_switch.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
glimit_switch.first()

In [ ]:
element_list_limit_hi_lo = group_element_list[group_element_list['SWM'] == 'S_LIMIT_HI_LO']
glimit_hi_lo = element_list_limit_hi_lo.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
glimit_hi_lo.first()
#print(element_list_limit_hi_lo.to_string())
#element_list_limit_hi_lo.count()

In [ ]:
element_list_leak = group_element_list[group_element_list['SWM'] == 'S_LEAK']
print(element_list_leak.to_string())

In [ ]:
#Particle counter modbus tcp
element_list_pms_modbus_tcp = group_element_list[group_element_list['SWM'] == 'S_PMS_MODBUS_TCP']
print(element_list_pms_modbus_tcp.to_string())


In [ ]:
#Air sampler modbus
element_list_aas_mas_tcp = group_element_list[group_element_list['SWM'] == 'S_AAS_MAS_TCP']
print(element_list_aas_mas_tcp.to_string())

SIS

In [ ]:
element_list_a_sis_pump = group_element_list[group_element_list['SWM'] == 'S_SIS']
ga_sis_pump = element_list_a_sis_pump.groupby(['SWM','SWM_Nr','Tag_Number_Definition'])
ga_sis_pump.first()

In [ ]:
#element_list_motors = element_list[element_list['SWM'] == 'S_MOTOR1']
#print(element_list_motors.to_string())

In [ ]:
#Group by SWM
#el_sortby_SWM = element_list.groupby(['SWM']).agg(list).copy()
#print(el_sortby_SWM.to_string())

In [ ]:
#Group by pdi number again
#this time
#Group by PID number
#element_list = element_list.groupby(['Prop_PID_Number']).agg(list).copy()

In [ ]:
#print(element_list.to_string())

In [ ]:
#list_1100 = element_list.loc[1.100].values.tolist()
#list_1100[0].sort()

In [ ]:
#print(list_1100[0])

end_new code

In [ ]:
#Remove duplicates - WARNING: It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um (maybe add exception when is an PCounter and duplicates elements)

#df_data_excel_uniques = df_data_excel.drop_duplicates()

In [ ]:
#df_data_excel_uniques.count()

In [ ]:
#gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
#gkk.first()

In [ ]:
#Get uniques values from the PID Elements (the columns for the matrix)
#df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
#df_unique_elememts.sort()
#print(df_unique_elememts)




#Export Unique elements to excel file - Once


In [ ]:
#Create a dataframe containing the elements
#pd_automation_code = pd.DataFrame(df_unique_elememts)
#Create the writer object
#automation_code = pd.ExcelWriter('AutomationCode.xlsx')
#Write dataframe to excel file
#pd_automation_code.to_excel(automation_code)
#Save excel 
#automation_code.save()
#Close excel
#automation_code.close()


In [ ]:
#Sort by PID number
#df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [ ]:
#Set PID number as multi-index (group by does better)
#df_data_excel_sorted.set_index(['Prop_PID_Number','Prop_PID_Element'])

In [ ]:
#df_data_excel_sorted.head(1000)

In [ ]:
#Add column, load data from Automation Code and mix
#df_data_excel_sorted['Automation Code'] = ''


In [ ]:
#df_data_excel_sorted

In [ ]:
#Load Automation Code excel file
#df_automation_code = pd.read_excel('AutomationCode.xlsx')
#df_automation_code.head()

In [ ]:
#Lookup
#df = df_data_excel
#df['Automation_Code'] = ''
#print (df)

In [ ]:
# Deprecated df['Automation Code'] = df.set_index('Prop_PID_Element').lookup(df_automation_code.Prop_PID_Element, df_automation_code.Automation_Code)

In [ ]:
#pd.melt(df,)
#print(df.combine_first(df_automation_code))

In [ ]:
#documents = [['SWM', 'SWM_NR'], ['CONN1', 'CONN1_NR'], ['CONN2', 'CONN2_NR'], ['CONN3', 'CONN3_NR'], 'AM', 'HW']
#documents = ['EL', 'AM', 'HW', 'FN']

In [ ]:
#
#df[df_unique_elememts] = pd.NaT
#documents = ['EL', 'AM', 'HW', 'FN']
#df[documents] = pd.NaT
#print(df.to_string())

ALARM MATRIX

In [ ]:
#df_am = pd.read_excel('AM.xlsm')

#df_am


In [ ]:
#df_am = pd.read_excel('AM.xlsm', sheet_name="Alarm Matrix")

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
#pd.set_option('display.colheader_justify', 'center')
#pd.set_option('display.precision', 3)

#display(df_am)